# 1.从html提取信息写入csv

In [ ]:
# 调用外部的py
from demo import *
hello()

# 2.将空缺的日期补齐

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
# First：找到没有进行创作的日子
# 1.读取文件 第一行为列名称
df=pd.read_csv('../csv绘图/info.csv',header=0,sep=',') #filename可以直接从盘符开始，标明每一级的文件夹直到csv文件，header=None表示头部为空，sep=' '表示数据间使用空格作为分隔符，如果分隔符是逗号，只需换成 ‘，’即可。
# 2.按日期进行升序排列
df2 = df.sort_values(by='day')
# 3.提取出 day列  将其转化为 yyyy/mm/dd的格式  将 str=2017/1/27 23:06  转化为  datetime类型
d1 = [datetime.strptime(x, '%Y/%m/%d %H:%M') for x in list(df2["day"])]
#       将 datetime 转化为 str 2017/01/27
d_2017 = [datetime.strftime(x, '%Y/%m/%d') for x in list(d1)]
# 4.创建全年的日子
all_2017=[datetime.strftime(x,'%Y/%m/%d') for x in list(pd.date_range(start='2017/01/01',  end="2017/12/31"))]
# 5.将上两项做差求出没有记录的日子
absent_2017 = [i for i in all_2017 if i not in d_2017]

# 6.将没有记录的日子进行补充  [day:当天,title:空,longitude,latitude,num:0]
dataframe = pd.DataFrame({'day': absent_2017,
                            'title': [None]*len(absent_2017),
                            'longitude': [None]*len(absent_2017),
                            'latitude': [None]*len(absent_2017),
                            'num': [0]*len(absent_2017)})
dataframe.to_csv("./info.csv",index=False, sep=',',mode='a', header=False)


In [ ]:
absent_2017 = ['2017/07/10','2017/07/12','2017/07/11']
absent_2017_np = [time.mktime(time.strptime(x, '%Y/%m/%d')) for x in absent_2017]

In [ ]:
from datetime import datetime
import time       
timeArray = time.strptime('2017/7/11 00:00', '%Y/%m/%d %H:%M')
#转换成时间戳
time.mktime(timeArray)     

In [ ]:
absent_2017_np

# 3.处理重复的日子

## 3.1 手动修改日期 并检查查看

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# First：找到没有进行创作的日子
# 1.读取文件 第一行为列名称
res_path = './data/我的抗战2.0_2020_info.csv'
# res_path = './data/2020_info.csv'
df=pd.read_csv(res_path,header=0,sep=',') 
# 手动修改 日期  
# 截取day的date————2017/02/02  是否唯一进行检查
M = [x.split(" ")[0] for x in df["day"].tolist()]
# 给原 df加一列 date
df['day_date'] = M
# 得到date重复的行  保留出现第一次的值  duplicated(keep=False)
day_date = df[df['day_date'].duplicated(keep=False)]
M = day_date[['day','title','day_date']]
print(len(M))
M

## 3.2自动整理日期
2019/08/19 12:00,,,,0,1566187200.0
2019/08/20 15:02,3.196  转文全部被拒  心情失落  又开始考虑复原,0,0,28,1566284520.0
2019/08/20 15:04,3.197  第一天便无法做到自律 

In [ ]:
def edit_info(res_path):
    # 手动修改 日期  
    # 截取day的date————2017/02/02 12:26  -->  2017/02/02  是否唯一进行检查
    df=pd.read_csv(res_path,header=0,sep=',') 
    M = [x.split(" ")[0] for x in df["day"].tolist()]
    # 给原 df加一列 date
    df['day_date'] = M
    # 得到date重复的行  保留出现第一次的值  duplicated(keep=False)
    day_date = df[df['day_date'].duplicated(keep=False)]
    M = day_date[['day','title','day_date']]
    print("重复的天数为：",len(M))
   
    all_index = M.index.tolist() 
      # 剔除 0 元素
    if 0 in all_index:
        all_index.remove(0)
    # 2.判断 index-1 是否为系统自动生成的 (num值是否为0)
    del_index = []

    for x in all_index:
        if ((df[x:x+1].day_date == '2020/02/13').bool()):
            print("2020/02/13")
        if ((df[x-1:x].num == 0).bool()):
            print("删除行",df[x-1:x].day)
            del_index.append(x-1)
            # df = df.drop([x-1])
        # 3.是——删掉 index-1这一行 
        # 4.修改 df[index] 的 date-1
            temp = df[x:x+1]
            dt = datetime.datetime.strptime(temp.at[temp.index.values[0],'day'],'%Y/%m/%d %H:%M')
            out_date = (dt + datetime.timedelta(days=-1)).strftime('%Y/%m/%d %H:%M')
            temp.day[x] = out_date
    # 5.更新csv文件
    df = df.drop(del_index)
    df.to_csv(res_path,index=False, sep=',')

## 3.3将没有坐标的日子进行临近填充

In [ ]:
# info_path = 'D:\MyJava\mylifeImg\others\echart\data\demo.csv'
# update_gps_info(info_path)
dir = 'D:\MyJava\mylifeImg\others\echart\data\\'
update_gps_dir(dir)

In [ ]:
# 检验填充后的数据是否还有 gps 为 0 或 Nan
import pandas as pd
import time
info_path = '../info.csv'
m=pd.read_csv(info_path,header=0,sep=',') 
day_date = m[(m['longitude'].isna()) | (m['longitude']=='0')]['day_date']
for x in day_date:
    if time.strptime(x, '%Y/%m/%d')<time.localtime():
        print(x)

In [ ]:
import pandas as pd
import requests
import json
import numpy as np
# 比较耗时 注意保存文件
# 添加一列  location_name  
def get_locationName(lng,lat):
    key = 'GjG3XAdmywz7CyETWqHwIuEC6ZExY6QT'
    r = requests.get(url='http://api.map.baidu.com/geocoder/v2/', params={'location':str(lat)+','+str(lng),'ak':key,'output':'json'})
    result = r.json()
    # print(result)
    province = result['result']['addressComponent']['province']
    city = result['result']['addressComponent']['city']
    district = result['result']['addressComponent']['district']
    street = result['result']['addressComponent']['street']
    street_number = result['result']['addressComponent']['street_number']
    return city+" "+district+" "+street+" "+street_number
    
info_path = 'D:\MyJava\mylifeImg\others\info.csv'
m=pd.read_csv(info_path,header=0,sep=',') 

location_list = []
for index, row in m.iterrows():
    if(np.isnan(row.longitude) or np.isnan(row.latitude)):
        location_list.append('无位置信息')
        continue
    location_name = get_locationName(row.longitude,row.latitude)
    location_list.append(location_name)
m["location_name"] = location_list
m.to_csv(info_path,index=False, sep=',')
# 单独写一个文件保存位置信息
m[["day","location_name"]].to_csv('./data/location_name.csv',index=False, sep=',')

In [ ]:
# 调用外部的py
from demo import *
classifyByYear(info_path,info_path)

## 3.4将文件按年进行分表

In [ ]:
# 1.读取文件
res_path = '../info.csv'
df=pd.read_csv(res_path,header=0,sep=',') 
# 2.按照day_date(2018/01/01)  列的前四位  2018进行分表
# day_date_list = list(df['day_date'])
day_date_list = [x.split('/')[0] for x in list(df['day_date'])]
year_list = list(set(day_date_list))
year_list = [eval(x) for x in list(set(day_date_list))]
# 3.将年份由小到大进行排序
year_list.sort()
# 4.分割年份  年份第一次出现的位置进行
all_years = [df[day_date_list.index(str(year_list[x])):day_date_list.index(str(year_list[x]+1))] for x in range(len(year_list)-1)]
# 最后一年
all_years.append(df[day_date_list.index(str(year_list[-1])):])
# 3.输出表
[df_year.to_csv("./data/"+str(year)+"_info.csv",index=False, sep=',')  for (year,df_year) in zip(year_list,all_years)]

# 4.地图视角

## 4.1 历史gps的热力图
var heatmapData = [{
	"lng": 116.191031,
	"lat": 39.988585,
	"count": 10
}, {
	"lng": 116.389275,
	"lat": 39.925818,
	"count": 11
}]

In [ ]:
# 1.制作生成形如上述的mapData.js文件，引入到html中即可
# 2.读取 csv文件的经纬度信息
import pandas as pd
# res_path = './data/2017_info.csv'
res_path = '../info.csv'
df=pd.read_csv(res_path,header=0,sep=',') 
# 3.对经纬度进行 计数统计
# df = df.tail(10)
m = df.groupby(['longitude','latitude']).count()

# 4.输出到js文件
# "lng": 116.191031, "lat": 39.988585, "count": 10
# 将 index 设置为列
m = m.reset_index()
df = pd.DataFrame(columns = ['lng','lat','count'])
df[['lng','lat','count']] = m[['longitude','latitude','day']]
heatmapData = "var heatmapData_local = " + df.to_json(orient = 'records')
# print(heatmapData)
# 打开一个文件
fo = open("./data/heatmapData_local.js", "w")
fo.write(heatmapData)
# 关闭打开的文件
fo.close()

## 4.2把日记标注在地图上

In [ ]:
import pandas as pd

# 1.制作生成形如上述的mapData.js文件，引入到html中即可
# 2.读取 csv文件的经纬度信息
def gen_diarymapData(filepath,desPath):
    df=pd.read_csv(filepath,header=0,sep=',') 
    # 3.将需要在地图上显示信息 day title longitude latitude输出到js文件
    d = pd.DataFrame(columns = ['lng','lat','day','title'])
    d[['lng','lat','day','title']] = df[['longitude','latitude','day','title']]
    # print(heatmapData)
    # 打开一个文件
    with open(desPath, 'w', encoding='utf-8') as file:
        diarymapData = "var diarymapData = " + d.to_json(orient = 'records',force_ascii=False)
        file.write(diarymapData)
        # 关闭打开的文件
        file.close()
res_path = '../info.csv'
desPath = './data/diarymapData.js'

## 4.3使用 leaflet timeline 进行滑动显示
按格式进行输出到文件
eqfeed_callback(
    {"type":"FeatureCollection",
        "metadata":{
        "generated":1630507840000,
        "title":"USGS All Earthquakes, Past Day","status":200,"api":"1.10.3","count":292},
        "features":[
                        {   "type":"Feature",
                            "properties":{
                                "mag":1,
                                "time":1609430400000,
                                "title":"title1"
                                },
                            "geometry":{
                                "type":"Point",
                                "coordinates":[-141.1344,60.2327]},
                            // "id":"ak021b7rquc6"
                            },
                        }
                    ],
        });

### 4.4.1 json格式的转换

In [ ]:
# 例子1
import json
df = pd.DataFrame(
    [['CompanyA',' Orange ' , 3000], 
    ['CompanyB',' Apple ' , 2000],
    ['CompanyB', ' Grapes ' , 1000],
    ],
    columns=["company","productName","price"],
)
m = df.groupby(['company'])['productName','price'].apply(lambda x: x.to_dict('r'))
m.to_json(orient="columns")

In [ ]:
#  例2
import json
df = pd.DataFrame(
    [['PnL', ' Company A',' Orange ' , 3000], 
    ['PnL', ' Company B',' Apple ' , 2000],
    ['PnL', ' Company B', ' Grapes ' , 1000],
    ['Tax', ' Company A',' Orange ' , 3000],
    ['Tax', ' Company B',' Apple ' , 2000],
    ['Tax', ' Company B',' Grapes ' , 1000]],
    columns=["type", "company","productName","price"],
)
d = (df.groupby(['type','company'])['productName','price']
       .apply(lambda x: x.to_dict('r'))
       .reset_index(name='data')
       .groupby('type')['company','data']
       .apply(lambda x: x.set_index('company')['data'].to_dict())
       .to_json())
df

In [ ]:
import time
import pandas as pd
res_path = './data/2021_info.csv'
# res_path = '../info.csv'


In [ ]:
# 1.制作生成形如上述的mapData.js文件，引入到html中即可
# 2.读取 csv文件的经纬度信息
def gen_traceData(res_path,desPath):
    df=pd.read_csv(res_path,header=0,sep=',') 
    # df = df[:100][['day','title','longitude','latitude']]
    df = df[['day','title','longitude','latitude']]
    # 查找包含空值的行
    # df[df.isnull().T.any()]
    # 删除包含空值的行
    df.dropna(axis=0, how='any', inplace=True)
    # 3.将需要在地图上显示信息 day title longitude latitude输出到json文件
    dicList = [] 
    time_list = [int(time.mktime(time.strptime(x, '%Y/%m/%d %H:%M'))*1000) for x in list(df.day)]
    for gtime,title,lng,lat,day in zip(time_list,list(df['title']),list(df['longitude']),list(df['latitude']),list(df['day'])):
        # print(time,title,lng,lat)
        dicList.append({
            "type":"Feature",
            "properties":{  
                            "mag":1,
                            "day":day,
                            "time":gtime,
                            "title":title,
                            },
            "geometry":{"type":"Point",
                            "coordinates":[lng,lat]},}
        )
    # 写进json文件
    eqfeed_callback =  {
            "type":"FeatureCollection",
            "metadata":{
            "generated":int(time.time()*1000),
            "title":"USGS All Earthquakes, Past Day","status":200,"api":"1.10.3","count":len(df)},
            "features":dicList}
    
    with open(desPath, 'w', encoding='utf-8') as file:
        traceData = "eqfeed_callback("+str(eqfeed_callback)+");"
        file.write(traceData)
        file.close()
res_path = '../info.csv'
desPath = './data/traceData.json'
gen_traceData(res_path,desPath)

# 5.html to markdown

In [ ]:
from natsort import natsorted
dir_name = "../我的抗战2.0"
dirList = os.listdir(dir_name)
htmlList = []
for f in dirList:
    if(f.endswith(".html")):
        htmlList.append(f)
# 按照数字开头进行排序 厉害
htmlList = natsorted(htmlList)
h_list = htmlList
year_html_list = [htmlList[0:370],htmlList[370:725],htmlList[725:1071],htmlList[1071:]]
h_lists = []
years = ['2018','2019','2020','2021']
for h_list,year in zip(year_html_list,years):
    h_list = [dir_name + "/" + x for x in h_list]
    h_lists.append(h_list)
    descpath = dir_name + "/" + year +".md"
    # htmls_to_md(h_list,descpath)

In [76]:
from markdownify import markdownify as md
def htmls_to_md(srcpath,descpath):
    res = ""
    # t0 = ""
    for path in srcpath:
        # 读取html格式文件
        with open(path, 'r', encoding='UTF-8') as f:
            htmlpage = f.read()
        # 处理html格式文件中的内容
            text = md(htmlpage)  
            text = text.replace("body, td {\n font-family: 微软雅黑;\n font-size: 10pt;\n }\n","")
            # 1.去除多余的换行
            text = text.replace("\n\n\n\n","")
            # 2.生成换行
            text = text.replace("  \n","  \n\n")
            # 3.去掉特殊字符
            text = text.replace("\xa0"," ")
            # 4.去掉标题  只去掉匹配的第一个
            text = re.sub(r'\n\n.*\n', "", text,1)
            # 5.二次转化视频标签
            text = gen_video_tag(text)
            res +=text
    with open(descpath, 'w', encoding='UTF-8') as f:
        f.write(res)
    # return res
srcpath = [h_lists[2][10]]
htmls_to_md(srcpath,"../我的抗战2.0/t.md")

In [75]:
# 二次转换视频
# 原始  <a href="dirname/v_name.mp4"> </a> 
# 经md转换后  [![1580400084168.mp4](4.11 8 h  声控游戏  刷线程进程基础研究语音控制  小黑屋午睡下不为例 二哥百果_files/07cc80e28ec6357bf86b517536b783e8.png)](4.11 8 h  声控游戏  刷线程进程基础研究语音控制  小黑屋午睡下不为例 二哥百果_files/1580400084168.mp4)
# 二次转化  <video src="dirname/v_name.mp4"></video>
import re
def gen_video_tag(s):
    return re.sub('(?P<value>\[.*?mp4\))', video_match, s)

def video_match(matched):
    value = matched.group('value')
    value = re.sub('(?P<value>\[.*\])', "\n", value)
    if value.endswith(".mp4)"):
        value = value.replace("(","<video src=\"").replace(")","\"></video>")
    return value
# s = "<div><a href=\"4.11 8 h  声控游戏  刷线程进程基础研究语音控制  小黑屋午睡下不为例 二哥百果_files/1580400084168.mp4\"><img src=\"4.11 8 h  声控游戏  刷线程进程基础研究语音控制  小黑屋午睡下不为例 二哥百果_files/07cc80e28ec6357bf86b517536b783e8.png\" alt=\"1580400084168.mp4\"></a></div>"
# s = md(s)
# s +=s
# # 提取 形如此类的字符串 (...)
# print(gen_video_tag(s))

In [ ]:
# h_list = htmlList[:10]
h_list = htmlList
dir_name = "../我的抗战1.0"
h_list = [dir_name + "/" + x for x in h_list]
descpath = dir_name + "/" + "kkk.md"
htmls_to_md(h_list,descpath)

In [ ]:
h_list